# ShipChecker

## Configuration

In [7]:
## raw data path
rawDataPath = './portData/rawData.txt'

## google search api path
googleSearchApiPath = '../googleSearchApi.txt'
## google search custom search engine id
googleSearchCseIdPath = '../googleSearchCSE.txt'

#Processingdata path
processingDataPath = './processingData.csv'

In [8]:
import pandas as pd
import warnings
import re
import requests
from bs4 import BeautifulSoup
import asyncio
import nest_asyncio

from pyppeteer import launch
from googleapiclient.discovery import build

from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))

## Load data

In [9]:
# load raw data and show the first 5 rows
rawData = pd.read_csv(rawDataPath, sep='\t')
rawData.head(5)

,DATE & TIME,ARRIVAL / DEPARTURE,VESSEL,VESSEL TYPE,FROM,TO,IN PORT
0,Wed 20 Mar 20:00,ARRIVAL,KOTA LIMA,CONTAINER SHIP (FULLY CELLULAR),CHIWAN PT,12 BROTHERSON DOCK (BD12),NO
1,Wed 20 Mar 22:00,ARRIVAL,MSC KANU F,CONTAINER SHIP (FULLY CELLULAR),BELL BAY,9 BROTHERSON DOCK (BD9),NO
2,Thu 21 Mar 03:00,ARRIVAL,WIDE INDIA,CONTAINER SHIP (FULLY CELLULAR),AUCKLAND,7 BROTHERSON DOCK (BD7),NO
3,Thu 21 Mar 15:00,ARRIVAL,GASCHEM HOMER,LPG TANKER,BRISBANE,BULK LIQUID BERTH 1 (BLB1),NO
4,Thu 21 Mar 17:00,ARRIVAL,MARI COUVA,CHEMICAL/PRODUCTS TANKER,ONSAN/ULSAN,BULK LIQUID BERTH 1 (BLB1),NO


In [10]:
# load google search api key and custom search engine id
googleApiKey = open(googleSearchApiPath, 'r').read()
print(googleApiKey[:5])
googleSearchCseId = open(googleSearchCseIdPath, 'r').read()
print(googleSearchCseId)

AIzaS
e0f84b16165d14dc2


In [11]:
# google search function and data extraction

api_key = googleApiKey
cse_id = googleSearchCseId

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res

def get_link(results):
    link = results['items'][0]['link']
    return link
    
def get_snippet(results):
    snippet = results['items'][0]['snippet']
    return snippet

def get_IMO_number(snippet):
    imo_number_match = re.search(r'IMO number, (\d+)', snippet)
    if imo_number_match:
        imo_number = imo_number_match.group(1)
    else:
        imo_number = None  
    return imo_number

def get_vessel_details(vessel_name):
    search_query = f"what is imo number for {vessel_name}"
    results = google_search(search_query, api_key, cse_id)
    link = get_link(results)
    snippet = get_snippet(results)
    imo_number = get_IMO_number(snippet)
    
    return link, snippet, imo_number

In [20]:
processingData = rawData.head(1)

processingData[['link','snippet','imo_number']] = processingData.apply(lambda row: get_vessel_details(row['VESSEL']), axis=1, result_type='expand')

processingData.to_csv(processingDataPath)

In [21]:

processingData

,DATE & TIME,ARRIVAL / DEPARTURE,VESSEL,VESSEL TYPE,FROM,TO,IN PORT,link,snippet,imo_number
0,Wed 20 Mar 20:00,ARRIVAL,KOTA LIMA,CONTAINER SHIP (FULLY CELLULAR),CHIWAN PT,12 BROTHERSON DOCK (BD12),NO,https://www.vesselfinder.com/vessels/details/9...,"IMO number, 9267651. Vessel Name, KOTA LIMA. S...",9267651


In [23]:
# import asyncio
# import nest_asyncio
# import pandas as pd
# from bs4 import BeautifulSoup
# from pyppeteer import launch

# nest_asyncio.apply()  # 应用nest_asyncio

# async def scrape(url):
#     browser = await launch(headless=True, args=['--no-sandbox'])
#     page = await browser.newPage()
#     await page.setUserAgent('My User Agent')
#     await page.goto(url)
#     await page.waitForSelector('#port-calls', {'timeout': 50000})  # 等待ID为port-calls的元素加载

#     # 获取ID为port-calls的元素的HTML内容
#     element = await page.querySelector('#port-calls')
#     content = await page.evaluate('(element) => element.innerHTML', element)

#     await browser.close()
#     return content

# def parse_html(html_content):
#     # 使用BeautifulSoup解析HTML
#     soup = BeautifulSoup(html_content, 'html.parser')

#     # 找到所有相关的div元素
#     divs = soup.find_all('div', class_='_1hgmG')

#     # 解析数据
#     data = []
#     for div in divs:
#         # 获取地点和到达时间
#         location = div.find_previous_sibling('a').text.strip()
#         arrival = div.find('div', class_='_1GQkK').text.strip()
#         data.append({'location': location, 'Arrival': arrival})

#     return data

# def reshape_data(data):
#     # 创建DataFrame
#     df = pd.DataFrame(data)

#     # 重塑DataFrame以使每个位置和到达成为单独的列
#     reshaped_df = pd.DataFrame()
#     for i, row in enumerate(df.itertuples(), 1):
#         reshaped_df[f'location{i}'] = [row.location]
#         reshaped_df[f'Arrival{i}'] = [row.Arrival]

#     return reshaped_df

# async def process_data(link):
#     url = link
#     html_content = await scrape(url)
#     data = parse_html(html_content)
#     reshaped_df = reshape_data(data)
#     return reshaped_df

# # 示例调用
# result_df = await process_data(processingData['link'])
# result_df


TypeError: Object of type Series is not JSON serializable

In [ ]:


nest_asyncio.apply()  # 应用nest_asyncio

async def scrape(url):
    browser = await launch(headless=True, args=['--no-sandbox'])
    page = await browser.newPage()
    await page.setUserAgent('My User Agent')
    await page.goto(url)
    await page.waitForSelector('#port-calls', {'timeout': 50000})  # 等待ID为port-calls的元素加载

    # 获取ID为port-calls的元素的HTML内容
    element = await page.querySelector('#port-calls')
    content = await page.evaluate('(element) => element.innerHTML', element)

    await browser.close()
    return content

url = 'https://www.vesselfinder.com/vessels/details/9267651'

# 在Jupyter notebook中运行异步代码
div_content = await scrape(url)

# 打印div中的内容
print(div_content)


<div><a href="/ports/AUBTB002" class="flx _rLk t5UW5"><span class="flag-icon _jFl3" title="Australia" style="background-image:url(https://static.vesselfinder.net/images/flags/4x3/au.svg)"></span>Botany Bay, Australia</a><div class="flx _1hgmG"><div class="_211eJ"><div class="_2nufK">Arrival (UTC)</div><div class="_1GQkK">Mar 20, 09:48</div></div><div class="_211eJ"><div class="_2nufK">Departure (UTC)</div><div class="_1GQkK">-</div></div><div class="_2q7TN"><div class="_2nufK">In Port</div><div class="_1GQkK">-</div></div></div></div><div><a href="/ports/CNSZX001" class="flx _rLk t5UW5"><span class="flag-icon _jFl3" title="China" style="background-image:url(https://static.vesselfinder.net/images/flags/4x3/cn.svg)"></span>Shenzhen, China</a><div class="flx _1hgmG"><div class="_211eJ"><div class="_2nufK">Arrival (UTC)</div><div class="_1GQkK">Mar 6, 21:03</div></div><div class="_211eJ"><div class="_2nufK">Departure (UTC)</div><div class="_1GQkK">Mar 7, 09:14</div></div><div class="_2q7TN

In [ ]:

# 假定 html_content 包含了你的HTML内容
html_content = div_content

# 使用BeautifulSoup解析HTML
soup = BeautifulSoup(html_content, 'html.parser')

# 找到所有相关的div元素
divs = soup.find_all('div', class_='_1hgmG')

# 解析数据
data = []
for div in divs:
    # 获取地点和到达时间
    location = div.find_previous_sibling('a').text.strip()
    arrival = div.find('div', class_='_1GQkK').text.strip()
    data.append({'location': location, 'Arrival': arrival})

# 创建DataFrame
df = pd.DataFrame(data)

# 重塑DataFrame以使每个位置和到达成为单独的列
reshaped_df = pd.DataFrame()
for i, row in enumerate(df.itertuples(), 1):
    reshaped_df[f'location{i}'] = [row.location]
    reshaped_df[f'Arrival{i}'] = [row.Arrival]

print(reshaped_df)


               location1       Arrival1        location2      Arrival2  \
0  Botany Bay, Australia  Mar 20, 09:48  Shenzhen, China  Mar 6, 21:03   

       location3      Arrival3        location4      Arrival4       location5  \
0  Nansha, China  Mar 5, 21:15  Shanghai, China  Mar 2, 18:57  Qingdao, China   

        Arrival5  
0  Feb 29, 10:28  
